# Importing Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import geocoder # import geocoder
from geopy.geocoders import Nominatim
import folium # map rendering library
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# _Part 1_

## Creating empty list to save 3 types of value
__td contains all the values of table__  
__removing \n__  
__breaking after getting all the required values__ 

In [2]:
p=[]
b=[]
n=[]
dict1={1:p,2:b,3:n}
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = bs(response.text, 'html.parser')
ind=1
for i in soup.findAll('td'):
    if i.text.replace('\n','')=='':
        break
    print(ind,i.text.replace('\n',''))
    dict1[ind].append(i.text.replace('\n',''))
    if(ind<3):
        ind+=1
    else:
        ind=1
    
df=pd.DataFrame(dict1)        
    

1 M1A
2 Not assigned
3 Not assigned
1 M2A
2 Not assigned
3 Not assigned
1 M3A
2 North York
3 Parkwoods
1 M4A
2 North York
3 Victoria Village
1 M5A
2 Downtown Toronto
3 Regent Park, Harbourfront
1 M6A
2 North York
3 Lawrence Manor, Lawrence Heights
1 M7A
2 Downtown Toronto
3 Queen's Park, Ontario Provincial Government
1 M8A
2 Not assigned
3 Not assigned
1 M9A
2 Etobicoke
3 Islington Avenue, Humber Valley Village
1 M1B
2 Scarborough
3 Malvern, Rouge
1 M2B
2 Not assigned
3 Not assigned
1 M3B
2 North York
3 Don Mills
1 M4B
2 East York
3 Parkview Hill, Woodbine Gardens
1 M5B
2 Downtown Toronto
3 Garden District, Ryerson
1 M6B
2 North York
3 Glencairn
1 M7B
2 Not assigned
3 Not assigned
1 M8B
2 Not assigned
3 Not assigned
1 M9B
2 Etobicoke
3 West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
1 M1C
2 Scarborough
3 Rouge Hill, Port Union, Highland Creek
1 M2C
2 Not assigned
3 Not assigned
1 M3C
2 North York
3 Don Mills
1 M4C
2 East York
3 Woodbine Heights
1 M5C
2 Downtown T

In [3]:
## The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df=df.rename(columns={1:'PostalCode',2:'Borough',3:'Neighborhood'})
df=df.replace('Not assigned',np.nan)
df.dropna(subset=['Borough'],inplace=True)

In [4]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.isna().sum()

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

In [6]:
print('Dataframe contains {} rows and {} columns'.format(df.shape[0],df.shape[1]))

Dataframe contains 103 rows and 3 columns


# _Part 2_

## Getting lat and long using geocoder google api

In [7]:
# # initialize your variable to None
# latitude=[]
# longitude=[]
# # loop until you get the coordinates
# for pc in df['PostalCode']:
#     lat_lng_coords = None
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(pc))
#         lat_lng_coords = g.latlng
#     latitude.append(lat_lng_coords[0])
#     longitude.append(lat_lng_coords[1])

In [8]:
## AS geocoder is not working. Trying out CSV
loc_df=pd.read_csv('Geospatial_Coordinates.csv')

In [9]:
df=pd.merge(df, loc_df, left_on='PostalCode',right_on='Postal Code')
df.drop('Postal Code',axis=1,inplace=True)

In [10]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Part 3

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df['Borough'].unique()),df.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Canada.

In [12]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of canada are 43.6534817, -79.3839347.


In [13]:
# create map of New York using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

#### Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'ZZWEDFHEJ3XRRUIFX114VECQL2MQBVWMVVEPARXM3210YT2M' # your Foursquare ID
CLIENT_SECRET = '3RUGSF0PKLLXI0XL0AVH3B1DBVBBUQL5UX1SAXJH2EH0P3YS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZZWEDFHEJ3XRRUIFX114VECQL2MQBVWMVVEPARXM3210YT2M
CLIENT_SECRET:3RUGSF0PKLLXI0XL0AVH3B1DBVBBUQL5UX1SAXJH2EH0P3YS


In [15]:
df.loc[1, 'Neighborhood']

'Victoria Village'

In [16]:
neighborhood_latitude = df.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Victoria Village are 43.725882299999995, -79.31557159999998.


In [17]:
# type your answer here
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZZWEDFHEJ3XRRUIFX114VECQL2MQBVWMVVEPARXM3210YT2M&client_secret=3RUGSF0PKLLXI0XL0AVH3B1DBVBBUQL5UX1SAXJH2EH0P3YS&v=20180605&ll=43.725882299999995,-79.31557159999998&radius=500&limit=100'

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4a97723649c121d4eda12b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bermondsey',
  'headerFullLocation': 'Bermondsey, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.7303823045, 'lng': -79.30935618239715},
   'sw': {'lat': 43.72138229549999, 'lng': -79.32178701760282}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c633acb86b6be9a61268e34',
       'name': 'Victoria Village Arena',
       'location': {'lat': 43.72348055545508,
        'lng': -79.31563520925143,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72348055545508,
          'lng': -79.315

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\shajinda\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Tim Hortons,Coffee Shop,43.725517,-79.313103
2,Portugril,Portuguese Restaurant,43.725819,-79.312785
3,Eglinton Ave E & Sloane Ave/Bermondsey Rd,Intersection,43.726086,-79.313620
4,Pizza Nova,Pizza Place,43.725824,-79.312860


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    
    return(nearby_venues)

In [23]:
Venues = getNearbyVenues(names=df['Neighborhood'],latitudes=df['Latitude'],longitudes=df['Longitude'])


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale


KeyError: 'groups'

In [ ]:
print(Venues.shape)
Venues.head()

In [ ]:
Venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(Venues['Venue Category'].unique())))

## Analyze Each Neighborhood

In [ ]:
# one hot encoding
onehot = pd.get_dummies(Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

In [ ]:
onehot.shape

In [ ]:
onehot_grouped = onehot.groupby('Neighborhood').mean().reset_index()
onehot_grouped

In [ ]:
onehot_grouped.shape

In [ ]:
num_top_venues = 5

for hood in onehot_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = onehot_grouped[onehot_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = onehot_grouped['Neighborhood']

for ind in np.arange(onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 3

onehot_grouped_clustering = onehot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(onehot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

data_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
data_merged = data_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

data_merged.dropna(subset=['Cluster Labels'],axis=1,inplace=True)
data_merged['Cluster Labels']=data_merged['Cluster Labels'].astype(int)

data_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Neighborhood'], data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters